<font size="6">Let's predict the new world champion..!</font>

'

In [74]:
import pandas as pd
import pickle 
from scipy.stats import poisson


<font size="3">1.Import the tables</font> 

In [75]:
tables=pickle.load(open('../Web_Scraping /Data/dict_table','rb'))
history=pd.read_csv('../Web_Scraping /Data/Clean_Fifa_worldcup_missing_data.csv')
fixture=pd.read_csv('../Web_Scraping /Data/Clean_Fixture_2022.csv')
history.rename({'HomeAway':'AwayTeam'},axis=1,inplace=True)

<font size="6">2.Calculate Team Strength..</font>

In [76]:
df_home=history[['HomeTeam','HomeGoals','AwayGoals']]
df_away=history[['AwayTeam','AwayGoals','HomeGoals']]
df_home=df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'})
df_away=df_away.rename(columns={'AwayTeam':'Team','HomeGoals':'GoalsConceded','AwayGoals':'GoalsScored'})

'

<font size="3">Concatenate df_home and df_away make a groupby by 'Team' and calculate the average..</font> 

In [77]:
df_team_strength=pd.concat([df_home,df_away],ignore_index=True).groupby('Team').mean()

<font size="3">This model was chosen because it fits the Poisson distribution</font> 

<img src="../source/Condiciones.png">

<font size="3">So.. how works?</font> 

<img src="../source/Distribucion de poisson.png">

<font size="3">Having understood the formula. let's apply it to our example!!</font> 

In [78]:
def predict_point(home,away):
    if home in df_team_strength.index and away in df_team_strength.index:
        #goals scored* goals conceded
        lamb_home=df_team_strength.at[home,'GoalsScored']*df_team_strength.at[away,'GoalsConceded']
        lamb_away=df_team_strength.at[away,'GoalsScored']*df_team_strength.at[home,'GoalsConceded']
        prob_home,prob_away,prob_draw=0,0,0
        for x in range(0,11):#Numbers of Goals of Home Team
            for y in range(0,11):#Numbers of Goals of Away Team
                p=poisson.pmf(x,lamb_home)*poisson.pmf(y,lamb_away)
                if x==y:
                    prob_draw+=p
                elif x>y:
                    prob_home+=p
                else:
                    prob_away+=p
        points_home=3*prob_home+prob_draw
        points_away=3*prob_away+prob_draw
        return (points_home,points_away)
    else:
        return(0,0)

<font size="3">Let's test the function! </font> 

In [79]:
predict_point('Bolivia','Argentina')

(0.010097814248235259, 2.894594744185821)

<font size="6">3.Predicting the world cup 2022</font>

<font size="3">3.1 Group stage</font> 

In [80]:
fixture_48=fixture[:48].copy()
fixture_knockout=fixture[48:56].copy()
fixture_quarter=fixture[56:60].copy()
fixture_semi=fixture[60:62].copy()
fixture_final=fixture[62:].copy()

In [81]:
for group in tables:
    teams_in_group = tables[group]['Team'].values
    df_fixture_group_6 = fixture_48[fixture_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_point(home, away)
        tables[group].loc[tables[group]['Team'] == home, 'Pts'] += points_home
        tables[group].loc[tables[group]['Team'] == away, 'Pts'] += points_away

    tables[group] = tables[group].sort_values('Pts', ascending=False).reset_index()
    tables[group] = tables[group][['Team', 'Pts']]
    tables[group] = tables[group].round(0)

<font size="3">3.2 knockout</font> 

In [82]:
fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [83]:
for group in tables:
    group_winner = tables[group].loc[0, 'Team']
    runners_up = tables[group].loc[1, 'Team']
    fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

fixture_knockout['winner'] = '?'
fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


<font size="3">And the winner of knockouts are..</font> 

In [84]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_point(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [85]:
get_winner(fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


<font size="3">3.3 Quarter Final</font> 

In [86]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [87]:
update_table(fixture_knockout,fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


<font size="3">And the Winner is..</font> 

In [88]:
get_winner(fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


<font size="3">3.4 Semi Final</font> 

In [89]:
update_table(fixture_quarter,fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


<font size="3">And the Winner is..</font> 

In [90]:
get_winner(fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


In [97]:

update_table(fixture_semi,fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


<font size="3">3.5 Final!!</font> 

<font size="3">And the Winner of  World Cup is....</font> 

In [98]:
get_winner(fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil
